These are the URLs for the JSON data powering the ESRI/ArcGIS maps.

In [13]:
few_crashes_url = 'http://www.arcgis.com/sharing/rest/content/items/5a8841f92e4a42999c73e9a07aca0c23/data?f=json&token=lddNjwpwjOibZcyrhJiogNmyjIZmzh-pulx7jPD9c559e05tWo6Qr8eTcP7Deqw_CIDPwZasbNOCSBHfthynf-8WRMmguxHbIFptbZQvnpRupJHSY8Abrz__xUteBS93MitgvoU6AqSN5eDVKRYiUg..'
removed_url = 'http://www.arcgis.com/sharing/rest/content/items/1e01ac5dc4d54dc186502316feab156e/data?f=json&token=lddNjwpwjOibZcyrhJiogNmyjIZmzh-pulx7jPD9c559e05tWo6Qr8eTcP7Deqw_CIDPwZasbNOCSBHfthynf-8WRMmguxHbIFptbZQvnpRupJHSY8Abrz__xUteBS93MitgvoU6AqSN5eDVKRYiUg..'

We need a way to easily extract the actual data points from the JSON. The data will actually contain multiple layers (really, one layer per `operationalLayer`, but multiple `operationalLayers`) so, if we pass a title, we should return the `operationalLayer` corresponding to that title; otherwise, just return the first one.

In [57]:
import requests
def extract_features(url, title=None):
    r = requests.get(url)
    idx = 0
    found = False
    if title:
        while idx < len(r.json()['operationalLayers']):
            for item in r.json()['operationalLayers'][idx].items():
                if item[0] == 'title' and item[1] == title:
                    found = True
                    break
            if found:
                break
            idx += 1
    try:
        return r.json()['operationalLayers'][idx]['featureCollection']['layers'][0]['featureSet']['features']
    except IndexError, e:
        return {}

few_crashes = extract_features(few_crashes_url)
all_cameras = extract_features(removed_url, 'All Chicago red light cameras')
removed_cameras = extract_features(removed_url, 'red-light-cams')
print 'Found %d data points for few-crash intersections, %d total cameras and %d removed camera locations' % (
    len(few_crashes), len(all_cameras), len(removed_cameras))

Found 193 data points for few-crash intersections, 195 total cameras and 25 removed camera locations


Now we need to filter out the bad points from few_crashes - the ones with 0 given as the lat/lon.

In [35]:
filtered_few_crashes = [
    point for point in few_crashes if point['attributes']['LONG_X'] != 0 and point['attributes']['LAT_Y'] != 0]

Now let's build a dictionary of all the cameras, so we can merge all their info.

In [61]:
cameras = {}
for point in all_cameras:
    label = point['attributes']['LABEL']
    if label not in cameras:
        cameras[label] = point
        cameras[label]['attributes']['Few crashes'] = False
        cameras[label]['attributes']['To be removed'] = False

Set the `'Few crashes'` flag to True for those intersections that show up in `filtered_few_crashes`.

In [62]:
for point in filtered_few_crashes:
    label = point['attributes']['LABEL']
    if label not in cameras:
        print 'Missing label %s' % label
    else:
        cameras[label]['attributes']['Few crashes'] = True

Set the `'To be removed'` flag to True for those intersections that show up in `removed_cameras`.

In [69]:
for point in removed_cameras:
    label = point['attributes']['displaylabel'].replace(' and ', '-')
    if label not in cameras:
        print 'Missing label %s' % label
    else:
        cameras[label]['attributes']['To be removed'] = True

Now I'm curious: how many camera locations have few crashes and were slated to be removed?

In [78]:
counter = {
    'both': {
        'names': [],
        'count': 0
    },
    'crashes only': {
        'names': [],
        'count': 0
    },
    'removed only': {
        'names': [],
        'count': 0
    }
}

for camera in cameras:
    if cameras[camera]['attributes']['Few crashes']:
        if cameras[camera]['attributes']['To be removed']:
            counter['both']['count'] += 1
            counter['both']['names'].append(camera)
        else:
            counter['crashes only']['count'] += 1
            counter['crashes only']['names'].append(camera)
    elif cameras[camera]['attributes']['To be removed']:
        counter['removed only']['count'] += 1
        counter['removed only']['names'].append(camera)

print '%d locations had few crashes and were slated to be removed: %s\n' % (
    counter['both']['count'], '; '.join(counter['both']['names']))
print '%d locations had few crashes but were not slated to be removed: %s\n' % (
    counter['crashes only']['count'], '; '.join(counter['crashes only']['names']))
print '%d locations were slated to be removed despite having reasonable numbers of crashes: %s' % (
    counter['removed only']['count'], '; '.join(counter['removed only']['names']))

12 locations had few crashes and were slated to be removed: Kimball-Lincoln-McCormick; Osceola-Touhy; Vincennes-111th; Western-Pratt; Pulaski-Montrose; Cicero-Stevenson NB; Cottage Grove-95th; Harlem-Northwest Highway; Cornell-57th; Ashland-Diversey; Halsted-63rd; Elston-LaPorte-Foster

61 locations had few crashes but were not slated to be removed: California-Devon; Central-Lake; Western-Van Buren; Broadway-Sheridan-Devon; Laramie-Madison; Pulaski-Lawrence; Canal-Roosevelt; Cicero-Diversey; Kedzie-47th; Halsted-North; Stony Island-89th; Western-Madison; Illinois-Columbus; Kedzie-Irving Park; Pulaski-Division; Central-Irving Park; Pulaski-Armitage; Kedzie-Armitage; Cicero-Armitage; Clark-Irving Park; Oak Park-Grand; Sacramento-Lake; Western-Chicago; Western-Cermak; Ashland-Lawrence; Damen-63rd; Wells-North; California-47th; Central-Chicago; Ontario-Kingsbury; Cicero-Harrison; Central-Diversey; Kilpatrick-Irving Park; Western-Addison; Western-71st; Pulaski-Diversey; Pulaski-North; Centr